In [1]:
import numpy as np
import pandas as pd
import streamlit as st
import csv
from collections import defaultdict
from numpy.random import default_rng as rng

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as fm
from os import path
from matplotlib.pyplot import figure
if 'Sarabun' not in [x.name for x in matplotlib.font_manager.fontManager.ttflist]:
    matplotlib.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
matplotlib.rc('font', family='Sarabun')

from rsi_modules import *

mm3_url         = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=1475905714"
publication_url = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=2125519370"
theses_url      = "https://docs.google.com/spreadsheets/d/1NrR5_OKyydIklO2UPBwHYdqI0Y_x1CaQxrie4OTFaDs/export?format=csv&gid=609352385"

F = pd.read_csv(mm3_url)
publication = pd.read_csv(publication_url).fillna("")
theses = pd.read_csv(theses_url).fillna("")
theses = theses.drop_duplicates(subset=['รหัส (นักศึกษา)'], keep='last')

In [2]:
def faculty_members():
    names =  ['นพ.สมเกียรติ ลีละศิธร',
    'พญ.วัชรา ริ้วไพบูลย์',
    'ศ.ดร.ทวี  เชื้อสุวรรณทวี',
    'รศ.ดร.อาดัม นีละไพจิตร',
    'รศ.ดร.ณัฏฐนียา โตรักษา',
    'ผศ.ดร.อารี ภาวสุทธิไพศิฐ',
    'ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล',
    'ผศ.ดร.เจนจิรา เจนจิตรวาณิช',
    'ดร.รติรส จันทรสมดี',
    'ดร.อิศวรา ศิริรุ่งเรือง',
    'ดร.ธรรม จตุนาม',
    'ดร.รุจิรา สงขาว',
    'ดร.ปรเมศวร์ บุญยืน',
    'ดร.ปกรณ์กิตติ์ ม่วงประสิทธิ์',
    'ดร.สุนันทา ขลิบทอง',
    'ดร.วรางคณา รัชตะวรรณ',
    'ดร.ญาณิศา เนียรนาทตระกูล',
    'ดร.สุภชาญ ตรัยตรึงศ์สกุล',
    'ดร.วิษณุ นิตยธรรมกุล',
    'พฤหัส ศุภจรรยา',
    'ราษฏร์ บุญญา',
    'กุลยา ไทรงาม',
    'ณัฐวิชญ์ ศุภสินธุ์',
    'พชร นิลมณี',
    'สิรินทรา ฤทธิเดช',
    'สร้อยทอง หยกสุริยันต์',
    'ภัทรานิษฐ สงประชา',
    'อาจารย์พิเศษ (ภายนอกม.มหิดล)',
    'อาจารย์พิเศษ (ภายนอกคณะแพทย์ฯรามา ม.มหิดล)']
    return names

In [3]:
F = pd.read_csv(mm3_url)
f = remove_empty_columns(F)
f = merge_multiple_academic_years(f)

t1,t2,courses_year = report_mm3(f)

/workspaces/academic-staff-workload/rsi_modules.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f.dropna(how='all', axis=1, inplace=True)
/workspaces/academic-staff-workload/rsi_modules.py:169: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f['academic_year'] = expand.values()


In [ ]:
# units = summarize_course_units(f)
# lec, courses_year = courses(f)
# courses_year = calculate_teaching_hours_per_year(courses_year)

In [4]:
names = defaultdict(list)
for aj,co,yr in zip(f['อาจารย์ผู้รับผิดชอบรายวิชา'],f['รหัสวิชา'],f['academic_year']):
    names[aj].append((co,yr))

In [5]:
course_owners = pd.DataFrame([names]).T.rename(columns={0:'owner_courses'}).reset_index().rename(columns={'index':'name'})

In [39]:
MEMBERS = pd.DataFrame(faculty_members()).rename(columns={0:'name'})
TEACH_TERM = pd.DataFrame(courses_year).fillna(0).reset_index()
TEACH_TERM.rename(columns={'index':'name'}, inplace=True)
Member_Teaching_hrs = MEMBERS.merge(TEACH_TERM, on='name', how='left').fillna(0)
Member_Teaching_hrs


,name,[2568]-เทอม 1,[2567]-เทอม 1,[2567]-เทอม 2,[2568]-เทอม 2
0,นพ.สมเกียรติ ลีละศิธร,0.0,0.0,0.0,0.000000
1,พญ.วัชรา ริ้วไพบูลย์,5.0,0.0,0.0,2.000000
2,ศ.ดร.ทวี เชื้อสุวรรณทวี,41.5,0.0,0.0,22.000000
3,รศ.ดร.อาดัม นีละไพจิตร,45.0,0.0,0.0,46.000000
4,รศ.ดร.ณัฏฐนียา โตรักษา,67.0,0.0,0.0,2.000000
5,ผศ.ดร.อารี ภาวสุทธิไพศิฐ,84.0,0.0,0.0,1.000000
6,ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล,57.0,0.0,0.0,30.000000
7,ผศ.ดร.เจนจิรา เจนจิตรวาณิช,78.0,0.0,0.0,10.833333
8,ดร.รติรส จันทรสมดี,33.5,0.0,0.0,50.333333
9,ดร.อิศวรา ศิริรุ่งเรือง,47.0,0.0,0.0,37.333333


In [38]:
course_code_owner = defaultdict(list)
for n,cw in zip(course_owners['name'],course_owners['owner_courses']):
    course_code = defaultdict(list)
    for c in cw:
        course_code[c[1]].append(c[0])
    course_code_owner[n] = course_code
course_code_owner = pd.DataFrame(course_code_owner).T.fillna('0').reset_index().rename(columns={'index':'name'})
course_code_owner = MEMBERS.merge(course_code_owner, on='name', how='left').fillna(0)
course_code_owner

,name,[2568]-เทอม 1,[2568]-เทอม 2,[2567]-เทอม 1,[2567]-เทอม 2
0,นพ.สมเกียรติ ลีละศิธร,0,0,0,0
1,พญ.วัชรา ริ้วไพบูลย์,0,0,0,0
2,ศ.ดร.ทวี เชื้อสุวรรณทวี,"[รมคพ 705 , รมคพ 721]",[รมคพ 707 ],0,0
3,รศ.ดร.อาดัม นีละไพจิตร,[ รมคพ ๕๐๒ ],[รมคพ 512 ],0,0
4,รศ.ดร.ณัฏฐนียา โตรักษา,"[รมคพ 706, รมคพ ๕๐๓ ]",0,0,0
5,ผศ.ดร.อารี ภาวสุทธิไพศิฐ,"[รมศห ๓๑๔ , รมศท ๑๑๓ ]",0,0,0
6,ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล,"[รมกพ 602, รมกพ 616, รมศษ ๑๐๓ ]",[รมกพ 509],0,0
7,ผศ.ดร.เจนจิรา เจนจิตรวาณิช,"[รมศห ๓๑๒, รมศห ๓๑๐ , รมศท ๑๐๗ ]",[รมศห ๓๑๕],0,0
8,ดร.รติรส จันทรสมดี,"[รมหศ 298, รมหศ ๓๒๖ , รมหศ ๔๘๘]","[รมคพ 726, รมหศ 374, รมหศ 375, รมหศ 471]",0,0
9,ดร.อิศวรา ศิริรุ่งเรือง,"[รมคพ 708, รมคพ ๕๓๑ ]","[รมคพ 505, รมคพ 506 ]",0,0


In [37]:
number_course_code_owner = defaultdict(list)
for c in course_code_owner.columns:
    if c != 'name':
        for row in course_code_owner[c]:
            if row != '0':
                number_course_code_owner[c].append(len(row))
            else:
                number_course_code_owner[c].append(0)
        # number_course_code_owner[c] = course_code_owner[c]
number_course_code_owner = pd.DataFrame(number_course_code_owner)
number_course_code_owner['name']= course_code_owner['name']

Member_Courses = MEMBERS.merge(number_course_code_owner, on='name', how='left').fillna(0)
Member_Courses

,name,[2568]-เทอม 1,[2568]-เทอม 2,[2567]-เทอม 1,[2567]-เทอม 2
0,นพ.สมเกียรติ ลีละศิธร,0.0,0.0,0.0,0.0
1,พญ.วัชรา ริ้วไพบูลย์,0.0,0.0,0.0,0.0
2,ศ.ดร.ทวี เชื้อสุวรรณทวี,2.0,1.0,0.0,0.0
3,รศ.ดร.อาดัม นีละไพจิตร,1.0,1.0,0.0,0.0
4,รศ.ดร.ณัฏฐนียา โตรักษา,2.0,0.0,0.0,0.0
5,ผศ.ดร.อารี ภาวสุทธิไพศิฐ,2.0,0.0,0.0,0.0
6,ผศ.ดร.ธีรศักดิ์ ศรีสุรกุล,3.0,1.0,0.0,0.0
7,ผศ.ดร.เจนจิรา เจนจิตรวาณิช,3.0,1.0,0.0,0.0
8,ดร.รติรส จันทรสมดี,3.0,4.0,0.0,0.0
9,ดร.อิศวรา ศิริรุ่งเรือง,2.0,2.0,0.0,0.0


In [ ]:
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
# df.to_csv('publication.csv')
# st.write(df)

In [ ]:
def classify_publication(publication):  

    NAMES = []
    
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n) 

    for t in publication.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: publication.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: publication.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: publication.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)

    name_paper = defaultdict(list)
    for name in NAMES:
        papers = []
        for i,(role,TCI,Q,conf_int,conf_nat) in enumerate(zip(publication[name],
                                                            publication['วารสารไทย'],
                                                            publication['วารสารนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับนานาชาติ'],
                                                            publication['ประชุมวิชาการระดับชาติ'])):
            if role in ['First Author', 'Corresponding Author']:
                papers+=[TCI+Q+conf_int+conf_nat]
        name_paper[name.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [','').replace(']','')] = papers

    categories = ['Q1','Q2','Q3','Q4','TCI1','TCI2','อื่นๆ']
    blanktable = pd.DataFrame(np.zeros((len(name_paper.keys()), len(categories)),dtype=int), columns=categories)
    blanktable['name']=name_paper.keys()


    for idx, (name, papers) in enumerate(name_paper.items()):
        for paper in papers:
            for cat in categories:
                if cat in paper:
                    blanktable.loc[idx, cat] += 1
    
    # Reorder columns with 'name' first
    blanktable = blanktable[['name'] + [col for col in blanktable.columns if col != 'name']]
    
    # blanktable.to_csv('publication_classified.csv')

    return blanktable

In [ ]:
def get_publications_by_author(publication):
    """
    Create a dictionary where:
    - Key: author name (First Author or Corresponding Author)
    - Value: list of tuples (year, title, publication_type)
    """
    
    # Find author columns
    NAMES = []
    for n in publication.columns:
        if 'ชื่ออาจารย์และสถานะผู้แต่งหนังสือ' in n:
            NAMES.append(n)
    
    # Rename journal/conference columns for easier processing
    pub_copy = publication.copy()
    for t in pub_copy.columns:
        if 'วารสารไทย (TCI1-TCI2)' in t: 
            pub_copy.rename(columns={t:'วารสารไทย'}, inplace=True)
        elif 'วารสารนานาชาติ (Q1-Q4 หรืออื่นๆ)' in t: 
            pub_copy.rename(columns={t:'วารสารนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับนานาชาติ' in t: 
            pub_copy.rename(columns={t:'ประชุมวิชาการระดับนานาชาติ'}, inplace=True)
        elif 'ประชุมวิชาการระดับชาติ' in t: 
            pub_copy.rename(columns={t:'ประชุมวิชาการระดับชาติ'}, inplace=True)
    
    author_publications = defaultdict(list)
    
    # Get year and title columns
    year_col = 'ปีที่ตีพิมพ์ พศ. (คศ.)'
    title_col = 'ชื่อเรื่อง'
    
    # Process each row
    for idx, row in pub_copy.iterrows():
        year = row.get(year_col, '')
        title = row.get(title_col, '')
        
        # Determine publication type (Q or TCI category)
        pub_type = 'อื่นๆ'
        for category in ['Q1', 'Q2', 'Q3', 'Q4', 'TCI1', 'TCI2']:
            combined_text = (str(row.get('วารสารไทย', '')) + 
                           str(row.get('วารสารนานาชาติ', '')) +
                           str(row.get('ประชุมวิชาการระดับนานาชาติ', '')) +
                           str(row.get('ประชุมวิชาการระดับชาติ', '')))
            if category in combined_text:
                pub_type = category
                break
        
        # Process each author
        for name_col in NAMES:
            role = row.get(name_col, '')
            if role in ['First Author', 'Corresponding Author','EIC']:
                if role == 'First Author': role = 'FA'
                elif role == 'Corresponding Author': role = 'CA'
                # Extract author name from column name
                author_name = name_col.replace('ชื่ออาจารย์และสถานะผู้แต่งหนังสือ [', '').replace(']', '')
                # Add publication tuple (year, title, type)
                author_publications[author_name].append((year, pub_type, role, title))
    
    return dict(author_publications)


# Export author_pubs to CSV
def export_author_publications_to_csv(author_pubs, filename):
    """
    Export author publications dictionary to CSV file
    Format: Author, Year, Publication Type, Title
    """
    rows = []
    for author, publications in author_pubs.items():
        for year, pub_type, role, title in publications:
            rows.append({
                'Author': author,
                'Year': year,
                'Publication Type': pub_type,
                'Role': role,
                'Title': title
            })
    
    df = pd.DataFrame(rows)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Exported {len(rows)} publications to {filename}")
    return df

# Test the function
author_pubs = get_publications_by_author(publication)
df_export = export_author_publications_to_csv(author_pubs, 'author_publications.csv')


In [ ]:
author_pubs

In [ ]:

# Export to CSV
df_export.head()


In [ ]:
st.title('ภาระงานของคณาจารย์ในสถาบันราชสุดาปีการศึกษา 2568')

st.subheader("1. ชั่วโมงสอนและจำนวนวิชาที่รับผิดชอบ")

academic_year = st.radio(
    "เลือกเทอมและปีการศึกษา",
    ["2568/1", "2568/2"],
)

if academic_year == "2568/1":
    start_date = '8/5/2025'
    end_date   = '9/19/2025'
elif academic_year == "2568/2":
    start_date = '12/12/2025'
    end_date   = '12/30/2025'

f = F[F['Timestamp'].between(start_date, end_date)]
t1,t2 = report_mm3(f)

st.subheader("2. ผลงานตีพิมพ์ทางวิชาการในแต่ละปี")
publication,df = get_publication(publication)
df = df.reset_index().rename(columns={'index':'ปี'})
st.write(df)

st.subheader("3. ภาระงานควบคุมวิทยานิพนธ์")
B = pd.DataFrame(get_thesis_workload_new(theses)).T
st.write(B)

st.subheader("4. ภาพรวมความก้าวหน้าวิทยานิพนธ์ของนักศึกษา")
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)
st.write(A)
st.write("หมายเหตุ:")
st.write("MA  = หส.ปริญญาโท สาขาคุณภาพชชีวิตคนพิการ")
st.write("MEd = หส.ปริญญาโท สาขาการศึกษาพิเศษ")
st.write("PhD = หส.ปรัชญาดุษฎีบัณฑิต สาขาคุณภาพชีวิตคนพิการ")

In [ ]:
N = get_thesis_progression_summary(theses)
A = pd.concat(N, axis=1)

In [ ]:
st.write(A)